# Reproducing primer on reproducibility

Getting python code in [Arribas-Bel _et al._ (2017)](http://doi.org/10.1007/978-3-319-50590-9_17) working in 2021. 

> Arribas-Bel, D., de Graaff, T. and Rey, S.J. (2017). Looking at John Snow’s Cholera map from the twenty first century: A practical primer on reproducibility and open science. _In_: Jackson, R and Schaeffer, P. (Eds.) _Regional Research Frontiers Vol. 2_ (pp. 283-306). Springer, Cham. doi: 10.1007/978-3-319-50590-9_17

In [ ]:
import seaborn as sns
import pandas as pd
import libpysal as lps
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

### Loading and exploring the data

In [ ]:
# Load point data
pumps = gpd.read_file('../../data/dani/Pumps.shp')
# Load building blocks
blocks = gpd.read_file('../../data/dani/polys.shp')
# Load street network
js = gpd.read_file('../../data/dani/streets_js.shp')

In [ ]:
#Check data structure
js.head()

In [ ]:
# Quickly plot the streets
js.plot(cmap='Set1')

In [ ]:
# Set up figure and axis
f, ax = plt.subplots(1, figsize=(9, 9))
blocks.plot(ax=ax, facecolor='0.9', linewidth=0)

# Quantile choropleth of deaths at the street level
js.plot(column='Deaths_dens', scheme='fisher_jenks', ax=ax, cmap='YlGn', legend=True)

# Plot pumps
xys = np.array([(pt.x, pt.y) for pt in pumps.geometry])
ax.scatter(xys[:, 0], xys[:, 1], marker='^', color='k', s=50)

# Remove axis frame
ax.set_axis_off()
# Change background color of the figure
f.set_facecolor('0.75')
# Keep axes proportionate
plt.axis('equal')
# Title
f.suptitle('Cholera Deaths per 100m.', size=30)
# Draw
plt.show()

In [ ]:
import spaghetti as sp

In [ ]:
#Load the network
ntw = sp.Network('../../data/dani/streets_js.shp')

In [ ]:
# Create the spatial weights matrix
w = ntw.contiguityweights(graph=False)
# Rename IDs to match those in the `segIdStr` column
w.remap_ids(js['segIdStr'])
# Row standardize the matrix
w.transform = 'R'

In [ ]:
w['s0-1']

## Spatial Lag

In [ ]:
js['w_Deaths_dens'] = lps.weights.lag_spatial(w, js['Deaths_dens'])

In [ ]:
js[['segIdStr', 'Deaths_dens', 'w_Deaths_dens']].head()

In [ ]:
w.neighbors['s0-1']

In [ ]:
# Note that we first index the table on the index variable
neigh = js.set_index('segIdStr').loc[w.neighbors['s0-1'], 'Deaths_dens']
neigh

In [ ]:
neigh.mean()

In [ ]:
js['Deaths_dens_std'] = (js['Deaths_dens'] - js['Deaths_dens'].mean()) / js['Deaths_dens'].std()

In [ ]:
js['w_Deaths_dens_std'] = lps.weights.lag_spatial(w, js['Deaths_dens_std'])

In [ ]:
js[['segIdStr', 'Deaths_dens', 'w_Deaths_dens','w_Deaths_dens_std']].head()